In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode

# データセットがCOCOフォーマットなら以下のコードは次の3行で置き換えることができる。
from detectron2.data.datasets import register_coco_instances



1.6.0 True
gcc (Ubuntu 5.5.0-12ubuntu1~16.04) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [4]:
register_coco_instances("fox_falco_train", {}, "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json", "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train")
register_coco_instances("fox_falco_val", {}, "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_val.json", "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_val")

AssertionError: Dataset 'fox_falco_train' is already registered!

In [50]:
from detectron2.modeling import build_model
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import detectron2
import math as mt

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode

from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import Boxes

import tqdm
import time

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # falco, fox, smallBF
cfg.MODEL.WEIGHTS = "/home/tappun/wip/12_10_backup/output/model_final.pth"
cfg.MODEL.DEVICE='cuda:0'

cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.65
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.65
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.65

model = build_model(cfg)

predictor = DefaultPredictor(cfg)

# image_name = "fox_left_line_nasi.png"
# image_name = "falco_no_line.png"

image_name = "line_ari_2.png"

image = cv2.imread("./" + image_name)

outputs = predictor(image)
# print(outputs)

#------------------------<基本座標情報>-------------------

height, width, channels = image.shape[:3]
print("画像サイズ")
print("width: " + str(width))
print("height: " + str(height))

print("ステージ左上座標")
sbf_x1 = int(outputs["instances"].pred_boxes[0].tensor[0][0].item())
sbf_y1 = int(outputs["instances"].pred_boxes[0].tensor[0][1].item())
print(sbf_x1, sbf_y1)

print("ステージ右下座標")
sbf_x2 = int(outputs["instances"].pred_boxes[0].tensor[0][2].item())
sbf_y2 = int(outputs["instances"].pred_boxes[0].tensor[0][3].item())
print(sbf_x2, sbf_y2)

print("-------")

print("フォックス左上座標")
fox_x1 = int(outputs["instances"].pred_boxes[1].tensor[0][0].item())
fox_y1 = int(outputs["instances"].pred_boxes[1].tensor[0][1].item())
print(fox_x1, fox_y1)

print("フォックス右下座標")
fox_x2 = int(outputs["instances"].pred_boxes[1].tensor[0][2].item())
fox_y2 = int(outputs["instances"].pred_boxes[1].tensor[0][3].item())
print(fox_x2, fox_y2)

print("フォックス中心座標")
fox_cx = int((fox_x1 + fox_x2)/2)
fox_cy = int((fox_y1 + fox_y2)/2)
print(fox_cx, fox_cy)

print("フォックスの横幅")
fox_width = fox_x2 - fox_cx
print(fox_width)

print("-------")

print("ファルコ左上座標")
falco_x1 =  int(outputs["instances"].pred_boxes[2].tensor[0][0].item())
falco_y1 = int(outputs["instances"].pred_boxes[2].tensor[0][1].item())
print(falco_x1, falco_y1)

print("ファルコ右下座標")
falco_x2 =  int(outputs["instances"].pred_boxes[2].tensor[0][2].item())
falco_y2 = int(outputs["instances"].pred_boxes[2].tensor[0][3].item())
print(falco_x2, falco_y2)

print("ファルコ中心座標")
falco_cx = int((falco_x1 + falco_x2)/2)
falco_cy = int((falco_y1 + falco_y2)/2)
print(falco_cx, falco_cy)

print("ファルコの横幅")
falco_width = falco_x2 - falco_cx
print(falco_width)
print("-------")

#------------------------</基本座標情報>-------------------

#------------------------<キャラ同士の距離>----------------

print("フォックスとファルコの中心間の距離")
foxc_falcoc_diff = mt.sqrt((falco_cx - fox_cx)**2 + (falco_cy - fox_cy)**2)
print(foxc_falcoc_diff)

print("フォックスxとファルコxは重なっているか？")
is_overwrap_fox_falco_x = foxc_falcoc_diff <= fox_width/2 + falco_width/2
print(is_overwrap_fox_falco_x)

# print("フォックス右中とファルコ左中の距離")
foxr_falcol_diff = mt.sqrt((falco_x1 - fox_x2)**2 + (falco_cy - fox_cy)**2)
# print(foxr_falcol_diff)

# print("フォックス左中とファルコ右中の距離")
foxl_falcor_diff = mt.sqrt((falco_x2 - fox_x1)**2 + (falco_cy - fox_cy)**2)
# print(foxl_falcor_diff)

print("フォックスファルコの近い側同士の距離")
print(min(foxr_falcol_diff, foxl_falcor_diff))
print("------")

#------------------------</キャラ同士の距離>----------------

#--------------------<ライン計算>-------------------------
print("ステージ左x座標")
print(sbf_x1)

print("ステージ右x座標")
print(sbf_x2)

print("フォックス左xとステージ左xの距離")
sbflx_foxlx_diff = (fox_x1 - sbf_x1)
print(sbflx_foxlx_diff)

print("フォックス右xとフォックス右xの距離")
sbfrx_foxrx_diff = (sbf_x2 - fox_x2)
print(sbfrx_foxrx_diff)

print("ファルコ左xとステージ左xの距離")
sbflx_falcolx_diff = (falco_x1 - sbf_x1)
print(sbflx_falcolx_diff)

print("ファルコ右xとステージ右xの距離")
sbfrx_falcorx_diff = (sbf_x2 - falco_x2)
print(sbfrx_falcorx_diff)

#どっちの崖に寄っているか？
#画像全体の横幅から, 崖の右端を引く
#崖の左端と大小を取ることで, どっちの崖か推測
#画像サイズが1280, 左の崖が300,右の崖が1280の場合 -> 1280-1280=0 < 300で左の崖?
#画像サイズが1280, 左の崖が0, 右の崖が900の場合 -> 1280 - 900 = 380 > 0 で右の崖?
if((width - sbf_x2) < sbf_x1):
    print("左の崖に寄っている")
    
    print("フォックス左ライン")
    foxl_line = fox_x1 - sbf_x1
    print(foxl_line)
    
    print("ファルコ左ライン")    
    falcol_line = falco_x1 - sbf_x1
    print(falcol_line)
    
    if (foxl_line < falcol_line):
        print("フォックスの左ラインが少ない")
    elif(falcol_line < foxl_line):
        print("ファルコの左ラインが少ない")
    
elif((width - sbf_x2) > sbf_x1):
    print("右の崖に寄っている")
    
    print("フォックス右ライン")
    foxr_line = sbf_x2 - fox_x2
    
    print("ファルコ右ライン")
    falcor_line = sbf_x2 - falco_x2
    
    if (foxr_line < falcor_line):
        print("フォックスの右ラインが少ない")
    elif(falcor_line < foxr_line):
        print("ファルコの右ラインが少ない")

#--------------</ライン計算>--------------
        

v = Visualizer(image[:, :, ::-1],
    metadata=MetadataCatalog.get("fox_falco_train").set(thing_classes=["Falco", "Fox", "SmallBattleField"]),
    scale=0.8,
    instance_mode=ColorMode.IMAGE )

v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

detected_image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
detected_image = cv2.cvtColor(detected_image, cv2.COLOR_RGB2BGR)

detected_image_name = "detected_" + image_name
print(detected_image_name)
cv2.imwrite(detected_image_name, detected_image)


画像サイズ
width: 1280
height: 720
ステージ左上座標
56 459
ステージ右下座標
1280 718
-------
フォックス左上座標
445 282
フォックス右下座標
608 539
フォックス中心座標
526 410
フォックスの横幅
82
-------
ファルコ左上座標
621 241
ファルコ右下座標
806 535
ファルコ中心座標
713 388
ファルコの横幅
93
-------
フォックスとファルコの中心間の距離
188.28967045486058
フォックスxとファルコxは重なっているか？
False
フォックスファルコの近い側同士の距離
25.553864678361276
------
ステージ左x座標
56
ステージ右x座標
1280
フォックス左xとステージ左xの距離
389
フォックス右xとフォックス右xの距離
672
ファルコ左xとステージ左xの距離
565
ファルコ右xとステージ右xの距離
474
左の崖に寄っている
フォックス左ライン
389
ファルコ左ライン
565
フォックスの左ラインが少ない
detected_line_ari_2.png


True